## Imports & Installs

In [ ]:
# pip install -q -U google-generativeai

In [ ]:
import pandas as pd
import csv
import json
from collections import defaultdict
from google.colab import drive
from google.colab import userdata
import os
import google.generativeai as genai
import datetime
from datetime import datetime
drive.mount('/content/drive')

Mounted at /content/drive


## Read and clean data

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/TLC Recommendation - MVP/tier1_questions.csv")

In [ ]:
!cp "/content/drive/MyDrive/TLC Recommendation - MVP/tier1_questions.csv" "tier1_questions.csv"

In [ ]:
# df[df['ID']=='xxyv7g05itg91g29zgxxyv7gq6ta23u0']

In [ ]:
# clean df
shared_interests_cols = [col for col in df.columns if col.startswith("What are some shared interests between you and them?")]
upcoming_events_cols = [col for col in df.columns if col.startswith("Do they have any upcoming events or milestones we should be aware of?")]

# Combine shared interests into an array
# df['interests'] = df[shared_interests_cols].apply(lambda row: [interest for interest in row if pd.notna(interest)], axis=1)
df['interests'] = df[shared_interests_cols].apply(lambda row: [interest for interest in row.tolist() if pd.notna(interest)], axis=1)
df['upcoming_event'] = df[upcoming_events_cols].apply(lambda row: [event for event in row if pd.notna(event)], axis=1)

df = df.drop(columns=shared_interests_cols + upcoming_events_cols)

df = df.rename(columns = {
    "ID":"id",
    "Relationship":"relationship",
    "Relationship length":"relationship_length",
    "What is their age group?":"age",
    "What is their gender?": "gender",
    "What is their location in the UK?": "location",
    "relationship":"gift_for"
    })

In [ ]:
df.head(4)

,id,relationship,relationship_length,age,gender,location,interests,upcoming_event
0,rtn74bxm3862g6wn2rtn74fmi3yjx03s,Romantic partner,1 - 3 years,18-24,Female,England,"[Traveling, Cooking/baking, Outdoor activities...","[Upcoming birthday, Anniversary, Graduation, J..."
1,xxyv7g05itg91g29zgxxyv7gq6ta23u0,Romantic partner,1 - 3 years,25-34,Female,England,"[Traveling, Fitness/working out, Arts and cult...","[Upcoming birthday, Anniversary]"
2,ikeyc300dtzndbrfk3o6ikeyc3v4kama,Romantic partner,1 - 3 years,25-34,Male,England,"[Traveling, Cooking/baking, Outdoor activities...",[None at the moment]
3,80nzmfffm9slvrxsukj80nzmfyk77438,Friend,3 - 5 years,25-34,Female,England,"[Cooking/baking, Watching movies/TV shows, Din...",[Wedding]


## Gemini Recommendations v1.0

In [ ]:
genai.configure(api_key=userdata.get('gemini_key'))
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

In [ ]:
def determine_categories(df):
    prompt = f"""
    You are a gift recommendation expert. Based on the provided recipient information, identify 3 suitable gift categories.

    Recipient Information:
    - ID: {df.id}
    - Age: {df.age}
    - Gender: {df.gender}
    - Location: {df.location}
    - Interests: {df.interests}
    - Upcoming Events: {df.upcoming_event}
    - Relationship: {df.relationship}

    Return the categories as a JSON array:
    ["Category1", "Category2", "Category3"]

    Ensure the categories are tailored to the recipient's profile and aligned with their interests and events.
    """
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

def generate_gift_ideas(df, categories):
    prompt = f"""
    You are a gift recommendation expert. Using the categories: {categories}, suggest 3 unique gifts or experiences per category from UK-based brands.

    Your suggestions should:
    1. Be relevant to the recipient's profile and interests.
    2. Include only the base domain for each store (e.g., 'thortful.com' not 'thortful.com/search?q=category')
    3. Provide reasons for suitability.
    4. Feel free to recommend other unique and interesting UK-based stores beyond the examples provided.

    Use the following JSON format for your response:

    {{
      "user_id": "{df.id}",
      "recommendations": [
        {{
          "Product": "Gift/Experience Name",
          "Category": "Gift/Experience Category",
          "Explanation": "Why this gift/experience is suitable",
          "Store": "thortful.com"
        }},
        ...
      ]
    }}

    Note:
    - Suggested UK stores include (but are not limited to):
      - thortful.com
      - prezzybox.com
      - funkypigeon.com
      - notonthehighstreet.com
      - virginexperiencedays.co.uk
  - Feel encouraged to suggest other cool, unique UK-based stores that match the gift categories and recipient's interests

  Present only the JSON response—no additional text or commentary.
    """
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

In [ ]:
def run_prompts(df):
    # Step 1: Determine categories
    categories = determine_categories(df)
    # Step 2: Generate gift ideas
    gift_ideas = generate_gift_ideas(df,categories)
    return gift_ideas

In [ ]:
sample = df.head(1) # one user only

In [ ]:
gemini_recommendations = pd.DataFrame(columns =['user_id', 'Product','Category', 'Explanation'])

for index, row in sample.iterrows():
    recommendations = run_prompts(row)
    print(f"Recommendations recieved for user_id {row['id']}")
    print(recommendations)

    ## everything here is to save it as csv / df for readability sake
    # recommendations = recommendations.replace("```", "").replace("json", "")
    # recommendations_json = json.loads(recommendations)
    # recommendation_df = pd.DataFrame(recommendations_json['recommendations'])
    # recommendation_df['user_id'] = recommendations_json['user_id']
    # recommendation_df = recommendation_df[['user_id', 'Product','Category', 'Explanation','Link', 'Rating']]
    # print(f"Appending recommendations for user_id {row['id']}")
    # gemini_recommendations = pd.concat([gemini_recommendations, recommendation_df], ignore_index=True)

Recommendations recieved for user_id rtn74bxm3862g6wn2rtn74fmi3yjx03s
```json
{
  "user_id": "rtn74bxm3862g6wn2rtn74fmi3yjx03s",
  "recommendations": [
    {
      "Product": "Hot Air Balloon Ride",
      "Category": "Experiences",
      "Explanation": "A unique and memorable experience offering stunning views and a sense of adventure.",
      "Store": "virginexperiencedays.co.uk"
    },
    {
      "Product": "Weekend Cooking Course",
      "Category": "Experiences",
      "Explanation": "Learn new culinary skills and enjoy a fun, interactive experience.",
      "Store": "airbnb.co.uk"
    },
    {
      "Product": "West End Theatre Tickets",
      "Category": "Experiences",
      "Explanation": "A classic and enjoyable experience, perfect for a night out.",
      "Store": "todaytix.com"
    },
    {
      "Product": "Custom Engraved Watch",
      "Category": "Personalized Tech & Accessories",
      "Explanation": "A timeless and personalized gift that shows you care.",
      "Store":

## Gemini Recommendations (Moments)

In [ ]:
import datetime
from datetime import datetime

In [ ]:
def generate_moment_specific_recommendations(df):
    prompt = f"""
    You are a gift recommendation specialist who focuses EXCLUSIVELY on milestone moments. Your expertise is creating deeply meaningful, occasion-specific gifts that commemorate life's significant moments.

    Recipient & Moment Details:
    - ID: {df.id}
    - Age: {df.age}
    - Gender: {df.gender}
    - Interests: {df.interests}
    - Relationship to Gifter: {df.relationship}
    - MILESTONE EVENT: {df.moment_type}
    - EVENT DATE: {df.moment_date}
    - Days Until Event: {(pd.to_datetime(df.moment_date) - pd.Timestamp.now()).days if pd.notna(df.moment_date) else 'Unknown'}

    Your task: Create 5 EXTRAORDINARY gift recommendations that will create a LASTING MEMORY of this {df.moment_type}.

    IMPORTANT GUIDELINES:

    1. Focus on SYMBOLIC and MILESTONE-MARKING gifts that:
       - Commemorate this specific life transition
       - Create lasting memories or keepsakes
       - Mark the significance of this particular milestone
       - Potentially become family heirlooms or treasured mementos

    2. Consider RITUAL and TRADITION:
       - Traditional gifts associated with this milestone in UK culture
       - Modern interpretations of traditional milestone gifts
       - Cultural ceremonies or customs associated with this milestone

    3. Think about TRANSFORMATIONAL gifts:
       - Items that help the recipient transition to their new stage/role
       - Experiences that mark this life passage
       - Items that acknowledge the recipient's journey to this point

    4. Include PERSONALIZATION options:
       - Custom engravings with milestone dates
       - Personalized milestone markers
       - Commissioned items specific to this achievement

    5. Consider TIME-SPECIFIC elements:
       - Future opening/viewing (e.g., time capsules, aged items)
       - Annual remembrance options
       - Growth-over-time gifts (e.g., investments, plants, collectible series)

    Provide recommendations in these categories:
    - SYMBOLIC KEEPSAKE: A physical item that symbolizes this milestone
    - EXPERIENTIAL MILESTONE MARKER: An experience that commemorates this achievement
    - TRADITIONAL MILESTONE GIFT: Something following cultural traditions for this milestone
    - PERSONALIZED COMMEMORATION: A highly customized item marking this specific achievement
    - FUTURE-FOCUSED MILESTONE GIFT: Something that grows in meaning or value over time

    Use this JSON format:

    {{
      "user_id": "{df.id}",
      "milestone_event": "{df.moment_type}",
      "event_date": "{df.moment_date}",
      "milestone_recommendations": [
        {{
          "Product": "Name of the milestone gift",
          "Gift_Type": "SYMBOLIC KEEPSAKE/EXPERIENTIAL MILESTONE MARKER/TRADITIONAL MILESTONE GIFT/PERSONALIZED COMMEMORATION/FUTURE-FOCUSED MILESTONE GIFT",
          "Explanation": "Detailed explanation of why this gift has SPECIAL SIGNIFICANCE for this specific life milestone",
          "Store": "Specific UK retailer or artisan who specializes in milestone gifts"
        }},
        ...
      ]
    }}

    MILESTONE-SPECIFIC CONSIDERATIONS:
    - BIRTHDAYS: Age milestone traditions (18th, 21st, 30th, etc.), birth stone/flower, "coming of age" items
    - ANNIVERSARIES: Traditional gifts by year (1st: paper, 5th: wood, 25th: silver, etc.), commemorative items
    - GRADUATIONS: Field of study keepsakes, achievement markers, professional transition items
    - NEW JOB: Career milestone markers, professional identity items, workplace transition gifts
    - RETIREMENT: Career commemoration, legacy items, new chapter beginnings
    - NEW HOME: Hearth & home traditions, housewarming customs, home blessing items
    - NEW BABY: Family milestone traditions, heirloom starter items, parenting transition gifts
    - ENGAGEMENT/WEDDING: Partnership symbols, union traditions, future-together items

    Focus only on MILESTONE-MARKING gifts that would be inappropriate for everyday occasions.
    """
    try:
        model = gemini_model
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating milestone-specific recommendations: {e}")
        return None

In [ ]:
moments_sample = sample.copy()
moments_sample.loc[:,'moment_type'] = "birthday"
moments_sample.loc[:,'moment_date'] = datetime(1995, 4, 17)
moments_sample

,id,relationship,relationship_length,age,gender,location,interests,upcoming_event,moment_type,moment_date
0,rtn74bxm3862g6wn2rtn74fmi3yjx03s,Romantic partner,1 - 3 years,18-24,Female,England,"[Traveling, Cooking/baking, Outdoor activities...","[Upcoming birthday, Anniversary, Graduation, J...",birthday,1995-04-17


In [ ]:
for index, row in moments_sample.iterrows():
    recommendations = generate_moment_specific_recommendations(row)
    print(f"Recommendations recieved for user_id {row['id']}")
    print(recommendations)

Recommendations recieved for user_id rtn74bxm3862g6wn2rtn74fmi3yjx03s
```json
{
  "user_id": "rtn74bxm3862g6wn2rtn74fmi3yjx03s",
  "milestone_event": "birthday",
  "event_date": "1995-04-17 00:00:00",
  "milestone_recommendations": [
    {
      "Product": "Antique Compass Necklace with Birthstone Accent",
      "Gift_Type": "SYMBOLIC KEEPSAKE",
      "Explanation": "An antique compass symbolizes embarking on a new journey and navigating life's path, fitting for the transition into early adulthood. The birthstone accent (April - Diamond) adds a personalized touch and represents enduring strength and love. It is a tangible reminder of your support as she explores the world and her own identity.",
      "Store": "Etsy (search for 'antique compass necklace birthstone' or a local antique jeweller)"
    },
    {
      "Product": "Private Hot Air Balloon Ride over the Cotswolds with Champagne Brunch",
      "Gift_Type": "EXPERIENTIAL MILESTONE MARKER",
      "Explanation": "A hot air balloon

## Gemini Recommendations v1.1

In [ ]:
genai.configure(api_key=userdata.get('gemini_key'))
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

In [ ]:
def determine_categories(df):
    prompt = f"""
    You are a gift recommendation expert. Based on the provided recipient information, identify 5 suitable gift categories.

    Recipient Information:
    - ID: {df.id}
    - Age: {df.age}
    - Gender: {df.gender}
    - Location: {df.location}
    - Interests: {df.interests}
    - Upcoming Events: {df.upcoming_event}
    - Relationship: {df.relationship}
    - Budget Preference: {df.budget if 'budget' in df else 'Not specified'}
    - Exclusions: {df.exclusions if 'exclusions' in df else 'None specified'}
    - Current Season: {datetime.now().strftime('%B')} (Consider seasonal relevance for UK)

    Return the categories as a JSON array with explanations:
    [
      {{
        "category": "Category1",
        "reasoning": "Brief explanation why this category fits"
      }},
      {{
        "category": "Category2",
        "reasoning": "Brief explanation why this category fits"
      }},
      {{
        "category": "Category3",
        "reasoning": "Brief explanation why this category fits"
      }},
      {{
        "category": "Category4",
        "reasoning": "Brief explanation why this category fits"
      }},
      {{
        "category": "Category5",
        "reasoning": "Brief explanation why this category fits"
      }}
    ]

    If there's a specific upcoming moment, prioritize categories that would be appropriate for that occasion.
    Consider seasonal factors (weather, holidays, activities) relevant to the UK for the current month.
    Tailor categories to the recipient's interests and lifestyle.
    Ensure categories align with any budget preferences indicated.
    Avoid any categories related to explicitly excluded items or interests.
    """
    try:
        model = gemini_model
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating categories: {e}")
        return None

def generate_gift_ideas(df, categories):
    prompt = f"""
    You are a gift recommendation expert. Using the categories: {categories}, suggest 3 unique gifts or experiences per category from UK-based brands.

    Recipient Information:
    - Age: {df.age}
    - Gender: {df.gender}
    - Interests: {df.interests}
    - Upcoming Events: {df.upcoming_event}
    - Budget Preference: {df.budget if 'budget' in df else 'Not specified'}
    - Exclusions: {df.exclusions if 'exclusions' in df else 'None specified'}

    Your suggestions should:
    1. Be relevant to the recipient's profile and interests
    2. Be appropriate for the specific upcoming moment if provided
    3. Consider seasonal relevance for the UK (current weather, upcoming holidays, seasonal activities)
    4. Include only the base domain for each store (e.g., 'thortful.com' not 'thortful.com/search?q=category')
    5. Provide reasons for suitability that reference the recipient's profile, moment, and seasonal factors
    6. Suggest gifts across these specific price ranges:
       - Budget-friendly: £10-50
       - Mid-range: £50-£150
       - High-range: £150-£300
       - Premium: £300-£1000
       - Splurge: £1000+
    7. Avoid any explicitly excluded items or interests
    8. Feel free to recommend other unique and interesting UK-based stores beyond the examples provided

    Use the following JSON format for your response:

    {{
      "user_id": "{df.id}",
      "current_season": "{datetime.now().strftime('%B')}",
      "recommendations": [
        {{
          "Product": "Gift/Experience Name",
          "Category": "Gift/Experience Category",
          "Price_Range": "Budget-friendly (£10-30)/Mid-range (£30-75)/Premium (£75+)",
          "Seasonal_Relevance": "How this gift fits the current UK season",
          "Explanation": "Why this gift/experience is suitable for this person, occasion, and season",
          "Store": "thortful.com"
        }},
        ...
      ]
    }}

    Note:
    - Suggested UK stores include (but are not limited to):
      - thortful.com
      - prezzybox.com
      - funkypigeon.com
      - notonthehighstreet.com
      - virginexperiencedays.co.uk
      - buyagift.co.uk
      - moonpig.com
      - etsy.com (UK sellers)
      - johnlewis.com
      - selfridges.com
      - lookfantastic.com
      - firebox.com
      - menkind.co.uk
      - fortnumandmason.com
    - Feel encouraged to suggest other cool, unique UK-based stores that match the gift categories and recipient's interests
    - Consider seasonal UK events (e.g., Chelsea Flower Show in May, Wimbledon in June/July, Edinburgh Festival in August)

    Present only the JSON response—no additional text or commentary.
    """
    try:
        model = gemini_model
        response = model.generate_content(prompt)
        return response.text
    except Exception as e:
        print(f"Error generating gift ideas: {e}")
        return None

In [ ]:
def run_prompts(df):
    # Step 1: Determine categories
    categories = determine_categories(df)
    # Step 2: Generate gift ideas
    gift_ideas = generate_gift_ideas(df,categories)
    return gift_ideas

In [ ]:
sample = df.head(1) # one user only
sample.loc[:,"budget"] = "Splurge"
sample.loc[:,"exclusions"] = [["Alcohol","Pets"]]
sample

<ipython-input-83-1b29ee1ada02>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[:,"budget"] = "Splurge"
<ipython-input-83-1b29ee1ada02>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[:,"exclusions"] = [["Alcohol","Pets"]]


,id,relationship,relationship_length,age,gender,location,interests,upcoming_event,budget,exclusions
0,rtn74bxm3862g6wn2rtn74fmi3yjx03s,Romantic partner,1 - 3 years,18-24,Female,England,"[Traveling, Cooking/baking, Outdoor activities...","[Upcoming birthday, Anniversary, Graduation, J...",Splurge,"[Alcohol, Pets]"


In [ ]:
for index, row in sample.iterrows():
    recommendations = run_prompts(row)
    print(f"Recommendations recieved for user_id {row['id']}")
    print(recommendations)

Recommendations recieved for user_id rtn74bxm3862g6wn2rtn74fmi3yjx03s
```json
{
  "user_id": "rtn74bxm3862g6wn2rtn74fmi3yjx03s",
  "current_season": "April",
  "recommendations": [
    {
      "Product": "Luxury Spa Break in the Cotswolds",
      "Category": "Travel Experiences",
      "Price_Range": "Premium (£75+)",
      "Seasonal_Relevance": "A relaxing indoor activity while UK weather is still variable in April.",
      "Explanation": "Combines her interest in travel with a relaxing experience, perfect for a birthday treat and escaping before graduation pressures increase. Cotswolds offer beautiful scenery.",
      "Store": "virginexperiencedays.co.uk"
    },
    {
      "Product": "Weekend Glamping Getaway",
      "Category": "Travel Experiences",
      "Price_Range": "Premium (£75+)",
      "Seasonal_Relevance": "April is a good time for glamping as it's before the summer crowds and (hopefully) the weather is mild enough.",
      "Explanation": "Combines her love for travel and 

### Gemini Recommendation v1.1 + EXA Search

In [ ]:
# pip install exa-py
from exa_py import Exa
exa = Exa(api_key = userdata.get('exa_key'))

In [ ]:
recom = json.loads(recommendations.replace("```", "").replace("json", ""))

In [ ]:
def search_web_for_recommendations(recommendations):
    search_results = {}
    for recommendation in recommendations['recommendations']:
        product = recommendation['Product']
        query = f"{product} site:uk"
        result = exa.search_and_contents(query, text={"max_characters": 1000})
        search_results[product] = [item.url for item in result.results[:5]]
    return search_results

In [ ]:
search_web_for_recommendations(recom)

{'Luxury Spa Break in the Cotswolds': ['https://www.telegraph.co.uk/travel/destinations/europe/united-kingdom/england/cotswolds/articles/the-best-cotswolds-spa-hotels/',
  'https://www.houseandgarden.co.uk/best-spa-hotels-cotswolds',
  'https://www.independent.co.uk/travel/uk/england/cotswolds/best-cotswolds-spa-hotels-a9277176.html',
  'https://www.cotswoldhouse.com/',
  'https://www.calcot.co/calcot-spa/'],
 'Weekend Glamping Getaway': ['https://www.wigwamholidays.com/',
  'https://love-glamping.co.uk/',
  'https://goglamping.net/',
  'https://www.campsites.co.uk/glamping',
  'https://www.canopyandstars.co.uk/glamping/'],
 'National Trust Membership': ['https://www.nationaltrust.org.uk/membership',
  'https://www.nationaltrust.org.uk/membership/gift-membership',
  'https://www.nationaltrust.org.uk/membership/life-membership',
  'https://www.nationaltrust.org.uk/membership/enquiries',
  'https://www.nationaltrust.org.uk/membership/enquiries/membership-faqs'],
 'Lululemon Align Legging

## Recommendations with Tier 2 data

In [ ]:
sample.head()

,id,relationship,relationship_length,age,gender,location,interests,upcoming_event
0,rtn74bxm3862g6wn2rtn74fmi3yjx03s,Romantic partner,1 - 3 years,18-24,Female,England,"[Traveling, Cooking/baking, Outdoor activities...","[Upcoming birthday, Anniversary, Graduation, J..."


In [ ]:
sample.loc[:,"upcoming_moment"] = "Moving to a new home"
sample

,id,relationship,relationship_length,age,gender,location,interests,upcoming_event,upcoming_moment
0,rtn74bxm3862g6wn2rtn74fmi3yjx03s,Romantic partner,1 - 3 years,18-24,Female,England,"[Traveling, Cooking/baking, Outdoor activities...","[Upcoming birthday, Anniversary, Graduation, J...",Moving to a new home


In [ ]:
def determine_categories(df):
    prompt = f"""
    You are a gift recommendation expert. Based on the provided recipient information, identify 3 suitable gift categories.

    Recipient Information:
    - ID: {df.id}
    - Age: {df.age}
    - Gender: {df.gender}
    - Location: {df.location}
    - Interests: {df.interests}
    - Upcoming Events: {df.upcoming_event}
    - Relationship: {df.relationship}
    - Upcoming Moment: {df.upcoming_moment}

    Return the categories as a JSON array:
    ["Category1", "Category2", "Category3"]

    Ensure the categories are tailored to the recipient's profile and aligned with their interests and events.
    """
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

def generate_gift_ideas(df, categories):
    prompt = f"""
    You are a gift recommendation expert. Using the categories: {categories}, suggest 3 unique gifts or experiences per category from UK-based brands.

    Your suggestions should:
    1. Be relevant to the recipient's profile and interests.
    2. Include only the base domain for each store (e.g., 'thortful.com' not 'thortful.com/search?q=category')
    3. Provide reasons for suitability.
    4. Feel free to recommend other unique and interesting UK-based stores beyond the examples provided.

    Use the following JSON format for your response:

    {{
      "user_id": "{df.id}",
      "recommendations": [
        {{
          "Product": "Gift/Experience Name",
          "Category": "Gift/Experience Category",
          "Explanation": "Why this gift/experience is suitable",
          "Store": "thortful.com"
        }},
        ...
      ]
    }}

    Note:
    - Suggested UK stores include (but are not limited to):
      - thortful.com
      - prezzybox.com
      - funkypigeon.com
      - notonthehighstreet.com
      - virginexperiencedays.co.uk
  - Feel encouraged to suggest other cool, unique UK-based stores that match the gift categories and recipient's interests

  Present only the JSON response—no additional text or commentary.
    """
    model = genai.GenerativeModel("gemini-1.5-flash")
    response = model.generate_content(prompt)
    return response.text

In [ ]:
def run_prompts(df):
    # Step 1: Determine categories
    categories = determine_categories(df)
    # Step 2: Generate gift ideas
    gift_ideas = generate_gift_ideas(df,categories)
    return gift_ideas

In [ ]:
gemini_recommendations = pd.DataFrame(columns =['user_id', 'Product','Category', 'Explanation'])

for index, row in sample.iterrows():
    recommendations = run_prompts(row)
    print(f"Recommendations recieved for user_id {row['id']}")
    print(recommendations)


Recommendations recieved for user_id rtn74bxm3862g6wn2rtn74fmi3yjx03s
```json
{
  "user_id": "rtn74bxm3862g6wn2rtn74fmi3yjx03s",
  "recommendations": [
    {
      "Product": "Hot Air Balloon Ride over the Cotswolds",
      "Category": "Experiences",
      "Explanation": "A memorable and luxurious experience offering stunning views.  Suitable for adventurous recipients who appreciate scenic beauty.",
      "Store": "virginexperiencedays.co.uk"
    },
    {
      "Product": "Weekend Cooking Course at a Michelin-Starred Restaurant",
      "Category": "Experiences",
      "Explanation": "Develop culinary skills with a professional chef. Ideal for food enthusiasts seeking a hands-on learning experience.",
      "Store": "masterclass.com"
    },
    {
      "Product": "Tickets to a West End Show",
      "Category": "Experiences",
      "Explanation": "A classic and enjoyable evening out. Suitable for theatre lovers and those who appreciate the arts.",
      "Store": "todaytix.com"
    },
  

# OpenAI x Exa AI to search

In [ ]:
pip install exa-py openai

In [ ]:
from openai import OpenAI
import openai
import requests
client = OpenAI(api_key=userdata.get('openai_key'))

# def chat_gpt(prompt):
#   response = client.chat.completions.create(
#       model="gpt-4o-2024-08-06",
#       messages=[{"role": "user", "content": prompt}]
#   )
#   return response.choices[0].message.content.strip()

In [ ]:
def determine_categories_openai(df):
    prompt = f"""
    You are a gift recommendation expert. Based on the provided recipient information, identify 3 suitable gift categories.

    Recipient Information:
    - ID: {df.id}
    - Age: {df.age}
    - Gender: {df.gender}
    - Location: {df.location}
    - Interests: {df.interests}
    - Upcoming Events: {df.upcoming_event}
    - Relationship: {df.relationship}

    Return the categories as a JSON array:
    ["Category1", "Category2", "Category3"]

    Ensure the categories are tailored to the recipient's profile and aligned with their interests and events.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    categories = response.choices[0].message.content
    return json.loads(categories)
    # return response


def generate_gift_ideas_openai(df, categories):
    prompt = f"""
    You are a gift recommendation expert. Using the categories: {categories}, suggest 3 unique gifts or experiences per category from UK-based brands.

    Your suggestions should:
    1. Be relevant to the recipient's profile and interests.
    2. Include only the base domain for each store (e.g., 'thortful.com' not 'thortful.com/search?q=category')
    3. Provide reasons for suitability.
    4. Feel free to recommend other unique and interesting UK-based stores beyond the examples provided.

    Use the following JSON format for your response:

    {{
      "user_id": "{df.id}",
      "recommendations": [
        {{
          "Product": "Gift/Experience Name",
          "Category": "Gift/Experience Category",
          "Explanation": "Why this gift/experience is suitable",
          "Store": "thortful.com"
        }},
        ...
      ]
    }}

    Note:
    - Suggested UK stores include (but are not limited to):
      - thortful.com
      - prezzybox.com
      - funkypigeon.com
      - notonthehighstreet.com
      - virginexperiencedays.co.uk
    - Feel encouraged to suggest other cool, unique UK-based stores that match the gift categories and recipient's interests

    Present only the JSON response—no additional text or commentary.
    """
    response = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.7
    )
    gift_ideas = response.choices[0].message.content
    return json.loads(gift_ideas)
    # return response


In [ ]:
def run_prompts_openai(df):
    # Step 1: Determine categories
    categories = determine_categories_openai(df)
    # Step 2: Generate gift ideas
    gift_ideas = generate_gift_ideas_openai(df,categories)
    return gift_ideas

In [ ]:
openai_recommendations = pd.DataFrame(columns =['user_id', 'Product','Category', 'Explanation'])
for index, row in sample.iterrows():
    recommendations = run_prompts_openai(row)
    print(f"Recommendations recieved for user_id {row['id']}")
    print(recommendations)

ChatCompletion(id='chatcmpl-B4Bitq8TwumaqvbtW63uAspgwk84R', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='["Travel Accessories", "Cooking/Baking Tools", "Fitness Equipment"]', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1740338075, model='gpt-4-0613', object='chat.completion', service_tier='default', system_fingerprint=None, usage=CompletionUsage(completion_tokens=16, prompt_tokens=250, total_tokens=266, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Recommendations recieved for user_id rtn74bxm3862g6wn2rtn74fmi3yjx03s
{'user_id': 'rtn74bxm3862g6wn2rtn74fmi3yjx03s', 'recommendations': [{'Product': 'Personalised Leather Travel Wallet', 'Category': 'Travel Accessories', 'Explanation': 'Perfect for the organized t

In [ ]:
recommendations

{'user_id': 'rtn74bxm3862g6wn2rtn74fmi3yjx03s',
 'recommendations': [{'Product': 'Personalised Leather Travel Wallet',
   'Category': 'Travel Accessories',
   'Explanation': 'Perfect for the organized traveler, this wallet can hold all important documents and cards. Being personalised, it adds a unique touch to the gift.',
   'Store': 'notonthehighstreet.com'},
  {'Product': "World's Smallest Portable Bluetooth Speaker",
   'Category': 'Travel Accessories',
   'Explanation': 'This tiny speaker is ideal for music-loving travellers who wish to enjoy their favourite tunes without carrying bulky equipment.',
   'Store': 'prezzybox.com'},
  {'Product': 'Scratch the World® Travel Map',
   'Category': 'Travel Accessories',
   'Explanation': "This gift is excellent for globetrotters, allowing them to scratch off the countries they've visited, creating a unique and personalised world map over time.",
   'Store': 'thortful.com'},
  {'Product': 'Personalised Baking Cookbook',
   'Category': 'Cook

In [ ]:
pip install exa-py

In [ ]:
# pip install exa-py
from exa_py import Exa
exa = Exa(api_key = userdata.get('exa_key'))

In [ ]:
def search_web_for_recommendations(recommendations):
    search_results = {}
    for recommendation in recommendations['recommendations']:
        product = recommendation['Product']
        query = f"{product} site:uk"
        result = exa.search_and_contents(query, text={"max_characters": 1000})
        search_results[product] = [item.url for item in result.results[:5]]
    return search_results

In [ ]:
search_web_for_recommendations(recommendations)

{'Personalised Leather Travel Wallet': ['https://www.willyswallets.com/',
  'https://www.theleatherbagco.com/',
  'https://www.penheaven.co.uk/personalised-leather-gifts/travel-accessories',
  'https://www.bowlandleather.co.uk/',
  'https://ginoferrari.co.uk/'],
 "World's Smallest Portable Bluetooth Speaker": ['https://www.usb2u.co.uk/tiny-mini-speakers.html',
  'https://www.hearospeaker.com/',
  'https://minirigs.co.uk/',
  'https://minispeakers.weebly.com/',
  'https://bittyboomers.com/'],
 'Scratch the World® Travel Map': ['https://luckies.co.uk/collections/brands-scratch-map%e2%84%a2',
  'https://www.mapsinternational.co.uk/wall-maps/shop-by-area/world-wall-maps/scratch-world-maps.html',
  'https://www.firefliesdesigns.co.uk/collections/scratch-maps',
  'https://venturewholesale.co.uk/maps-international-scratch-the-world.html'],
 'Personalised Baking Cookbook': ['http://www.createcookbooks.co.uk/',
  'https://littlebearstudios.co.uk/products/personalised-baking-recipe-book?srsltid=

In [ ]:
# updating the query prompt and character limit to see if it improves output
def search_web_for_recommendations(recommendations):
    search_results = {}
    for recommendation in recommendations['recommendations']:
        product = recommendation['Product']
        query = f"{product} reviews OR best rated OR recommended site:uk" #maybe parse location here
        result = exa.search_and_contents(query, text={"max_characters": 3000})
        search_results[product] = [item.url for item in result.results[:5]]
    return search_results


In [ ]:
search_web_for_recommendations(recommendations)

{'Personalised Leather Travel Wallet': ['https://www.penheaven.co.uk/personalised-leather-gifts/travel-accessories',
  'https://www.elitetravelblog.com/trove-swift-and-cash-wrap-wallet-review/',
  'http://www.beautykinguk.co.uk/2015/10/david-hampton-leather-wallet.html',
  'https://www.willyswallets.com/',
  'https://www.penheaven.co.uk/personalised-leather-gifts/leather-wallets-personalised'],
 "World's Smallest Portable Bluetooth Speaker": ['https://www.usb2u.co.uk/tiny-mini-speakers.html',
  'https://willstocks.co.uk/review-doss-soundbox-portable-touch-bluetooth-speaker/',
  'https://www.cultofandroid.com/9785/dbests-mini-bluetooth-speakers-tiny-sound-grenades-review/',
  'https://www.youtube.com/watch?v=F0u-n3yNtpE',
  'https://www.whathifi.com/reviews/tribit-stormbox-micro-2'],
 'Scratch the World® Travel Map': ['https://www.findmeagift.co.uk/gifts/scratch-map.html',
  'https://www.rockytravel.net/blog/luckies-of-london-scratch-map-review/',
  'https://scratchmaps.net/product/worl

## Testing Recommendations output

In [ ]:
def score_recommendations(recommendations, recipient_interests, exa_search_results):
    scored_recs = []
    for rec in recommendations['recommendations']:
        product = rec['Product']
        interest_score = sum([interest.lower() in rec['Explanation'].lower() for interest in recipient_interests])
        search_score = len(exa_search_results.get(product, []))
        store_score = 1 if rec['Store'] in ['notonthehighstreet.com', 'prezzybox.com', 'virginexperiencedays.co.uk'] else 0.5

        total_score = (interest_score * 0.5) + (search_score * 0.3) + (store_score * 0.2)
        scored_recs.append((total_score, rec))

    # Sort by highest score
    scored_recs.sort(reverse=True, key=lambda x: x[0])
    return [rec for _, rec in scored_recs]

In [ ]:
def ensure_diversity(recommendations, max_per_store=2, max_per_category=2):
    store_counts = {}
    category_counts = {}
    diverse_recommendations = []

    for rec in recommendations:
        store = rec['Store']
        category = rec['Category']

        store_counts[store] = store_counts.get(store, 0) + 1
        category_counts[category] = category_counts.get(category, 0) + 1

        if store_counts[store] <= max_per_store and category_counts[category] <= max_per_category:
            diverse_recommendations.append(rec)

    return diverse_recommendations


# EXA example (old)

In [ ]:

# Step 1: Generate gift ideas with OpenAI
openai.api_key = userdata.get('openai_key')
client = OpenAI(api_key=userdata.get('openai_key'))

response = client.chat.completions.create(
    model="gpt-4",
    messages=[
        {"role": "system", "content": "You are a gift recommendation assistant."},
        {"role": "user", "content": "Suggest a birthday gift for a 25-year-old who loves photography."}
    ]
)

gift_idea =  response.choices[0].message.content

# Step 2: Search for gifts using EXA Web API
from exa_py import Exa
exa = Exa(api_key = "2ea304cd-e09e-47eb-9ae8-d9272b479ffa")
result = exa.search_and_contents(
    f"buy {gift_idea} online",
    text = { "max_characters": 1000 }
)
print(result)

Title: Alan Photo: Camera Store Singapore Affordable Photographic
URL: https://alanphoto.com.sg/
ID: https://alanphoto.com.sg/
Score: 0.17994411289691925
Published Date: 2024-05-22T00:00:00.000Z
Author: 
Image: https://alanphoto.com.sg//image/cache/catalog/Alan%20Photo%20Logo%20_RGB_-01-300x300.jpg
Favicon: https://alanphoto.com.sg//image/catalog/Alan Photo Logo _RGB_-01.jpg
Extras: None
Subpages: None
Text: Ever since Alan Photo Trading’s establishment in 1986, Alan Photo Trading has been committed to provide you with affordable photographic products and reliable services. Being in the industry for over 3 decades now, we have grown and understood the importance in keeping up with times. We serve photographers and videographers of all levels. Professional, amateurs or entirely new to the industry? We promise to provide you with the best solutions for your needs. Alanphoto is a Photographic industry as one of the top independent photographic camera store in Singapore, and have serving t

In [ ]:
# pip install exa-py
from exa_py import Exa
exa = Exa(api_key = "2ea304cd-e09e-47eb-9ae8-d9272b479ffa")
result = exa.search_and_contents(
    f"buy {gift_idea} online",
    text = { "max_characters": 1000 }
)
print(result)

Title: Login
URL: https://www.thecompellingimage.com/gifts
ID: https://www.thecompellingimage.com/gifts
Score: 0.16133329272270203
Published Date: 2023-02-06T12:23:59.000Z
Author: 
Image: None
Favicon: None
Extras: None
Subpages: None
Text: With courses ranging from beginner through aspiring professional levels, you’re sure to find just the right online and interactive TCI course for the photographer on your gift-giving list. Choose from two-, four-, or six-lesson courses, with or without an optional 30-minute instructor consultation call on Skype. Portfolio reviews and one-on-one mentorships are available, too. 
Here’s how it works. If you’re not already a TCI acount holder, you’ll be prompted to sign up for free. You’re then set to place your order. Choose the course category (two-, four-, six-lesson, portfolio review or one-on-one mentorship) you wish to purchase, make your secure payment using Pay Pal or a major credit card (Stripe Gateway), and select the date you’d like the recip

In [ ]:
gift_idea = response.choices[0].message.content
# Step 2: Search for gifts using EXA Web API
exa_api_key = userdata.get('exa_key')
query = f"buy {gift_idea} online"

exa_response = requests.get(
    f"https://api.exa.ai/search",
    params={"q": query, "key": exa_api_key}
)

exa_results = exa_response.json()

# Step 3: Combine and display results
recommendation = {
    "gift_idea": gift_idea,
    "search_results": exa_results
}

print(recommendation)


JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# pip install exa-py openai
from exa_py import Exa
from openai import OpenAI

exa = Exa(api_key='2ea304cd-e09e-47eb-9ae8-d9272b479ffa')
openai = OpenAI(api_key=userdata.get('openai_key'))

results = exa.search_and_contents(
    "Personalized Photo Album with handwritten captions", text = True)
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "user", "content": f"""Based on these
      search results, what are key developments in
      quantum computing? Context: {results}"""}
    ]
)

print(response.choices[0].message.content)

In [ ]:
# pip install exa-py
from exa_py import Exa
exa = Exa(api_key = "2ea304cd-e09e-47eb-9ae8-d9272b479ffa")
result = exa.search_and_contents(
    "Engraved Jewellery with a Special Date",
    text = { "max_characters": 1000 }
)
print(result)

Title: What is AGI? - Chatbots Life
URL: https://blog.chatbotslife.com/what-is-agi-438150f80356?gi=9afb14855d32
ID: https://blog.chatbotslife.com/what-is-agi-438150f80356?gi=9afb14855d32
Score: 0.22163283824920654
Published Date: 2017-10-30T00:47:28.000Z
Author: Peter Voss
Image: None
Favicon: https://miro.medium.com/v2/resize:fill:256:256/1*XMhZ5IeFYnQCahAtoj70bQ.png
Extras: None
Subpages: None
Text: Let’s start at the beginning. Why do we even need this term? 60 years ago when the term ‘AI’ was coined, the ambition was to build machines that can learn and reason like humans. Over several decades of trying and failing (badly), the original vision was largely abandoned. Nowadays almost all AI work relates to narrow, domain-specific, human-designed capabilities. Powerful as these current applications may be, they are limited to their specific target domain , and have very narrow (if any) adaptation or interactive learning ability. Most computer scientists graduating after the mid 80’s o